In [4]:
# Env
import os
import pandas as pd

os.chdir("/home/akikawa/repos/probe_designer/dataset/2023.10.24_ChunhongZheng_pancreatic_cancer copy")

In [5]:
barcode_df = pd.read_excel(r"../PRISM_30plex_barcode.xlsx", index_col=0)
binding_df = pd.read_excel(r"./gene_binding_site.xlsx")
probe_df = pd.DataFrame()

In [6]:
cont = 0
prism = 1
pre_gene_name = binding_df["gene_name"][0]
for num, gene in enumerate(binding_df["gene_name"]):
    if pre_gene_name != gene:
        pre_gene_name = gene
        cont = 0
        prism += 1
    elif cont == 3:
        continue
    # print(num, gene, prism)
    cont += 1
    binding = binding_df["binding"].loc[num]
    binding_l = binding[:20].lower()
    binding_r = binding[20:].lower()
    barcode = barcode_df.loc[f"Prism_{prism}", "Barcode (82bp)"]
    probe = binding_r + barcode + binding_l

    probe_info = pd.DataFrame(
        {
            "PRISM": [f"PRISM_{prism}"],
            "gene":[f'{gene}_{cont}'],
            "probe": [probe],
            "barcode": [barcode],
            "binding": [binding],
        },
    )

    if len(probe_df) == 0:
        probe_df = probe_info
    else:
        probe_df = pd.concat([probe_df, probe_info])

probe_df = probe_df.set_index('PRISM')
probe_df.to_excel(r'./probe.xlsx')